# Imports

In [1]:
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.callback.fp16 import *
from fastai.callback.wandb import *

import pandas as pd
import numpy as np

from efficientnet_pytorch import EfficientNet
import albumentations
import wandb

# Setup

In [2]:
class Config:
    testing     = False # must be same as create-folds.ipynb
    image_size  = 512
    batch_size  = 4
    epochs      = 1
    f_epochs    = 1
    train_folds = ['f0', 'f1']
    arch        = 'efficientnet-b4'
    
cfg = Config()

In [3]:
path_str = '../data'

images_path = Path(path_str + '/train_images')
csv_path = Path(path_str + '/train.csv')
folds_path = Path('folds.csv')

full_df = pd.read_csv(csv_path)
folds_df = pd.read_csv(folds_path)

# drop the first row so we get a even number for our folds
full_df.drop(full_df.head(1).index, inplace=True)

# Create a test dataset

In [4]:
if cfg.testing:
    full_df = full_df.head(120)
else:
    wandb.login(key="11b470b697ff94b3896d2243b147d42177a5cb7a")
    wandb.init(project="cassava", entity="teo03")

len(full_df)

wandb: Offline run mode, not syncing to the cloud.
wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` to enable cloud syncing.


21396

# Augmentation and train functions

In [5]:
class AlbumentationsTransform(RandTransform):
    split_idx,order = None, 2
    
    def __init__(self, train_aug, valid_aug): 
        store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)


def get_train_aug(size): 
    return albumentations.Compose([
            albumentations.RandomResizedCrop(size,size),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

def get_valid_aug(size): 
    return albumentations.Compose([
        albumentations.CenterCrop(size, size, p=1.),
        albumentations.Resize(size, size),
], p=1.)

In [6]:
def train(dls, fold):
    
    model = EfficientNet.from_pretrained(cfg.arch, num_classes=5)

    # define learner
    learn = Learner(
        dls=dls,
        model=model,
        opt_func=ranger,
        metrics=accuracy,
        loss_func=LabelSmoothingCrossEntropy(),
        cbs=[#WandbCallback(log_preds=False, log_model=False, n_preds=10)
    ]).to_fp16()
    
    lr_min, lr_steep = learn.lr_find(show_plot=False)
    print(f'found lr of({lr_min}): {round(lr_min, 5)}')
    
    
    # start model training
    learn.fine_tune(
        cfg.epochs,
        base_lr=round(lr_min, 5),
        freeze_epochs=cfg.f_epochs
    )
    
    learn.export(Path(f'model-{fold}.pkl'))
    
    return learn

In [7]:
def get_x(row): return images_path/row['image_id']
def get_y(row): return row['label']

# Training
train models on different folds of our data

In [8]:
for fold in cfg.train_folds:
    val_index = folds_df[fold].to_numpy()
    
    print(f'started training on {fold}')
    
    train_block = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_x=get_x,
        get_y=get_y,
        splitter=IndexSplitter(val_index),
        item_tfms= [
            AlbumentationsTransform(
                get_train_aug(size=cfg.image_size),
                get_valid_aug(size=cfg.image_size)
            )
        ],
        batch_tfms=[Normalize.from_stats(*imagenet_stats)]
    )

    dls = train_block.dataloaders(full_df, bs=cfg.batch_size)
    learn = train(dls, fold)

print(f'training on {cfg.train_folds} done')

started training on f0 /n
Loaded pretrained weights for efficientnet-b4


found lr of(0.0015848932787775993): 0.00158


epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 